In [ ]:
import sys
import os 

sys.path.append(os.path.abspath('../src'))  

In [ ]:
import pandas as pd
import numpy as np
import metrics as M
import viz as v
from scipy import stats 
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests

In [ ]:
df = pd.read_csv("../data/health_study_dataset.csv")

In [ ]:
statistics = M.statistik(df)
print("Statistics:")
statistics

In [ ]:
v.hist_blodtryck(df)

In [ ]:
v.box_sex_weight(df)

In [ ]:
v.bar_smokers(df)

In [ ]:
counts = M.sick_ones(df)
print(f"Andelen personer som har en sjukdom:" , counts) 

In [ ]:
simulated_disease, sim_dis, disease_count,people_quantity = M.disease_sim(df)
print(f"Verklig andel sjuka i datasetet: {disease_count:.3f}")
print(f"Simulerad andel sjuka: {simulated_disease:.3f}")
print(f"Skillnaden: {simulated_disease - disease_count:.3f}")


In [ ]:
lo, hi, mean_normal = M.ci_systolic_bp(df)
blo, bhi, bmean = M.ci_systolic_bp_bootstrap(df, B=3000)
true_mean = M.true_mean(df)
print(f"95% CI med normalapproximation: [{lo:.2f} , {hi:.2f}, {mean_normal:.2f}]")
print(f"95% CI med bootstrapmetoden: [{blo:.2f}, {bhi:.2f}, {bmean:.2f}]")
print(f"True mean: {true_mean:.2f}")


In [ ]:
lo, hi, mean_normal = M.ci_systolic_bp(df)
blo, bhi, bmean = M.ci_systolic_bp_bootstrap(df, B=3000)

skillnad_low = (blo - lo)
skillnad_high = (bhi - hi) 
print(f"Skillanden mellan metoderna: Low: {skillnad_low:.2f}")
print(f"Skillanden mellan metoderna: High: {skillnad_high:.2f}")


In [ ]:
rokare, ickerokare, t_stat, p_value, t_stat_w, p_value_w = M.hypotesprövning(df)
print(f"Standard t-test: t = {t_stat:.3f}, p = {p_value:.3f}")
print(f"Welch    t-test: t = {t_stat_w:.3f}, p = {p_value_w:.3f}")


## Resultat - Hypotesprövning ”Rökare har högre medel-blodtryck än icke-rökare.”: 
 I denna hypotesprövningen visade t-testet att det finns inte signikant skillnad i systolisk blodtryck mellan båda grupperna (rökare och ickerökare). 

Standard t-test: t = 0.461, p = 0.6450

Welch    t-test: t = 0.450, p = 0.6527

Hypotesen stöds inte av datan, då p-värdena är större än 0,05 och det finns ingen signifikant skillnad mellan grupperna.

In [ ]:
np.random.seed(42)
power_simulations = 1000
significant_count = 0


n_rok = len(rokare)
n_icke = len(ickerokare)

mean_rok = rokare.mean()
mean_icke = ickerokare.mean()
std_rok = rokare.std(ddof=1)
std_icke = ickerokare.std(ddof=1)

for _ in range(power_simulations):
    sim_rok = np.random.normal(loc=mean_rok, scale=std_rok, size=n_rok)
    sim_icke = np.random.normal(loc=mean_icke, scale=std_icke, size=n_icke)
    t_stat, p_val = stats.ttest_ind(sim_rok, sim_icke, equal_var=False)
    if p_val < 0.05:
        significant_count += 1

power = significant_count / power_simulations
print("Simulerad Testets power:", power)



## Metodval - motivering: 
- Konfidensintervall: normalapproximation eller bootstrap: 
Jag valde normalapproximation över bootstrap eftersom normalapproximation fungerar bra när urvalet är relativt stort, som i mitt fall. Bootstrap är mer flexibel och kan användas även för små urval eller när fördelningen är okänd, men är mer komplex att implementera.

- Hypotesprövning: bootstrap eller t-test funktion: 
Jag valde T-test funktion, eftersom den är snabb och enkel att använda. Eftersom mitt urval är ungefär normalfördelad, då är t-test det mest lämpliga alternativet för att den är enklast och mest effektiv.

Källor: 
- https://www.ne.se/uppslagsverk/encyklopedi/l%C3%A5ng/normalapproximation#:~:text=normalapproximation%2C%20inom%20statistiken%20metod%20att,ungef%C3%A4r%20normalf%C3%B6rdelade%20i%20enlighet%20med 

- https://sv.wikipedia.org/wiki/T-test

- https://science.nu/amne/val-av-statistisk-test-analys/#:~:text=One%2Dsample%20t%2Dtest&text=Testet%20anv%C3%A4nds%20om%20man%20vill,allts%C3%A5%20%C3%A4r%20ett%20hypotetiskt%20v%C3%A4rde).